In [0]:
%run ./01-config

In [0]:
# Set up environment widget and get value
try:
    dbutils.widgets.text("Environment", "dev", "Set the current environment")
except Exception:
    pass

ENV = dbutils.widgets.get("Environment").strip().lower()

if ENV not in ("dev", "prod"):
    raise ValueError("env must be 'dev' or 'prod'")

# Set storage account per environment
STORAGE_ACCOUNT = {
    "dev":  "sttfldevuks",
    "prod": "sttflproduks",
}[ENV]

# Set base zone URLs
data_zone_url = f"abfss://tfl-{ENV}@{STORAGE_ACCOUNT}.dfs.core.windows.net/data-zone"
checkpoint_zone_url = f"abfss://tfl-{ENV}@{STORAGE_ACCOUNT}.dfs.core.windows.net/checkpoint-zone"

# Set derived paths
raw_url = f"{data_zone_url}/raw"
test_data_url = f"{data_zone_url}/test_data"
checkpoint_url = f"{checkpoint_zone_url}/checkpoints"

# Bootstrap infrastructure

# Create catalog
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")

# Create schemas
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_landing}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_bronze}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_silver}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_gold}")

# Create external volumes
spark.sql(f"CREATE EXTERNAL VOLUME IF NOT EXISTS {raw_volume}  LOCATION '{raw_url}'")
spark.sql(f"CREATE EXTERNAL VOLUME IF NOT EXISTS {checkpoint_volume} LOCATION '{checkpoint_url}'")
if ENV != "prod":
    spark.sql(f"CREATE EXTERNAL VOLUME IF NOT EXISTS {test_volume} LOCATION '{test_data_url}'")

# Check accessibility of expected raw folders
for name, path in source_subdir.items():
    try:
        dbutils.fs.ls(path + "/")
        print(f"✅ ok: {name} -> {path}")
    except Exception as e:
        print(f"⚠️ not accessible yet: {name} -> {path} | {e}")

print("Bootstrap complete.")